In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import timeit
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import sys
lib = __import__("ihm_lib") #Provided by [1]

''' Map/resample TS'''
def map_ts(index, data, Hours):
    X = data[index]
    X_hat = pd.DataFrame(np.full([Hours.shape[0],len(X.columns)], np.nan))
    X_hat.columns = X.columns
    X_hat['Hours'] = pd.Series(Hours)
    mapping = [[min(range(len(X_hat)), key=lambda i: abs(X_hat['Hours'][i]-element))] for element in X['Hours']]
    mapping = np.array([mapping[idx][0] for idx in range(len(mapping))])
    unique_mapping = np.unique(mapping)
    indices = [np.where(mapping==element)[0] for element in unique_mapping]
    indices = dict(zip(unique_mapping, indices))
    keys = np.array(list(indices.keys()))
    for key,values in indices.items():
        for i in range(1, X.shape[1]):
            X_hat.iloc[key,i] = np.mean(X.iloc[values,i])
    return X,X_hat,pd.isna(X_hat),indices

In [9]:
path = 'Path to Train and Test IHM Pickle Files after reading'
with open(path +'train_ihm.pickle', 'rb') as handle:
    train = pickle.load(handle)
with open(path + 'test_ihm.pickle', 'rb') as handle:
    test = pickle.load(handle)
train = [train[idx].iloc[:,[0,2,9,11,12,13,14]] for idx in range(len(train))]
test = [test[idx].iloc[:,[0,2,9,11,12,13,14]] for idx in range(len(test))]
Hours = np.round(np.arange(1, 49 , dtype=float))
train_data = [ map_ts(index, train, Hours) for index in range(len(train))]
test_data = [ map_ts(index, test, Hours) for index in range(len(test))]
with open('final_train_ihm.pickle', 'wb') as handle:
    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('final_test_ihm.pickle', 'wb') as handle:
    pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)